In [ ]:
!pip install q scipy==1.1.0  #use this version of scipy  for sparse matrix operations

     |████████████████████████████████| 31.2MB 103kB/s 
ERROR: umap-learn 0.4.3 has requirement scipy>=1.3.1, but you'll have scipy 1.1.0 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement scipy==1.4.1; python_version >= "3", but you'll have scipy 1.1.0 which is incompatible.
ERROR: plotnine 0.6.0 has requirement scipy>=1.2.0, but you'll have scipy 1.1.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1


In [ ]:
import numpy as np
import torch 
import matplotlib.pyplot as plt
import torchvision
from time import time 
from torchvision import datasets, transforms
from torch import nn,optim 
from scipy.sparse.linalg import lsqr
from scipy.sparse import csr_matrix
from scipy.sparse import random
from scipy.sparse import rand
from scipy import stats


In [ ]:

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)

#set random seed for reproducibility
torch.manual_seed(0)

#define the model 
inputsize = 3072
hiddensizes = [128,128,128]
outputsize = 10

criterion = nn.NLLLoss()
images, labels = next(iter(trainloader))
images = images.view(images.shape[0],-1)


net = nn.Sequential(nn.Linear(inputsize,hiddensizes[0]),nn.ReLU(),nn.Linear(hiddensizes[0],hiddensizes[1]),nn.ReLU(),
                    nn.Linear(hiddensizes[1],hiddensizes[2]),nn.ReLU(),nn.Linear(hiddensizes[2],outputsize),nn.LogSoftmax(dim=1))
net.apply(init_weights)

w_init= list(net.parameters())
torch.save(net.state_dict(),'weights_i.pth')

print(net)
logps = (net(images))
loss = criterion(logps,labels)


optimizer = optim.SGD(net.parameters(),lr = 0.003, momentum = 0.9 )


Sequential(
  (0): Linear(in_features=3072, out_features=128, bias=True)
  (1): ReLU()
  (2): Linear(in_features=128, out_features=128, bias=True)
  (3): ReLU()
  (4): Linear(in_features=128, out_features=128, bias=True)
  (5): ReLU()
  (6): Linear(in_features=128, out_features=10, bias=True)
  (7): LogSoftmax()
)


In [ ]:
#train
epochs =15
time0 = time()
for e in range(epochs):
    running_loss = 0
    for images,labels in trainloader:
        images = images.view(images.shape[0],-1)
        optimizer.zero_grad()
        output = net(images)
        loss = criterion(output,labels)
        loss.backward()
        optimizer.step()
        running_loss+=loss.item()
#     else:
    print("Epoch {} - Training loss: {}".format(e, running_loss/len(trainloader)))
#     print(running_loss)
            
    print("\n Training time in minutes = ", (time()-time0)/60)

Epoch 0 - Training loss: 1.7369819321507216

 Training time in minutes =  0.8147761106491089
Epoch 1 - Training loss: 1.5908039342612028

 Training time in minutes =  1.6542569955190023
Epoch 2 - Training loss: 1.535713394139558

 Training time in minutes =  2.499661421775818
Epoch 3 - Training loss: 1.5014891576437652

 Training time in minutes =  3.369001011053721
Epoch 4 - Training loss: 1.4648741655728221

 Training time in minutes =  4.262768618265787
Epoch 5 - Training loss: 1.4385393301317095

 Training time in minutes =  5.222326683998108
Epoch 6 - Training loss: 1.4246769202800096

 Training time in minutes =  6.1326057394345606
Epoch 7 - Training loss: 1.3989324913316965

 Training time in minutes =  7.012182048956553
Epoch 8 - Training loss: 1.3943393396295607

 Training time in minutes =  7.897658546765645
Epoch 9 - Training loss: 1.3783520725233853

 Training time in minutes =  8.798887185255687
Epoch 10 - Training loss: 1.3626846667224168

 Training time in minutes =  9.7

In [ ]:
#testing

correct_count, all_count = 0, 0
for images,labels in testloader:
    for i in range(len(labels)):
        img = images[i].view(1, 3072)
        with torch.no_grad():
            logps = net(img)
        ps = torch.exp(logps)
        probab = list(ps.numpy()[0])
        pred_label = probab.index(max(probab))
        true_label = labels.numpy()[i]
        if(true_label == pred_label):
              correct_count += 1
        all_count += 1
print("Number Of Images Tested =", all_count)
print("\nModel Accuracy =", (correct_count/all_count))
model_accuracy = correct_count/all_count
np.save("model_accuracy_3.npy",model_accuracy)

Number Of Images Tested = 10000

Model Accuracy = 0.4617


In [ ]:
torch.save(net.state_dict(),'weights_f.pth')

#Projecting weights on affine spaces defined by the set P: {AW = AW_final} 
#Projection is done using L2 minimization w= argmin ||w-w_initial|| in P

#vectorize W
wf =torch.load('weights_f.pth')
wf1 = np.asarray(wf['0.weight'].resize(1,hiddensizes[0]*3072))
wf2 = np.asarray(wf['2.weight'].resize(1,hiddensizes[0]*hiddensizes[1]))
wf3 = np.asarray(wf['4.weight'].resize(1,hiddensizes[1]*hiddensizes[2]))
wf4 = np.asarray(wf['6.weight'].resize(1,hiddensizes[2]*10))
wf_vec = np.concatenate((wf1,wf2,wf3,wf4),axis =1)
wf_vec = wf_vec.T
dim_w = wf_vec.shape[0]

/usr/local/lib/python3.6/dist-packages/torch/tensor.py:365: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


In [ ]:
#128 node architecture
#A
from scipy import sparse
n_runs =2
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384,32768,65536,131072,262144,427264])
den = 10/dim_w
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))
rvs = stats.norm().rvs

wi = torch.load('weights_i.pth')
wi1 = np.asarray(wi['0.weight'].resize(1,hiddensizes[0]*3072))
wi2 = np.asarray(wi['2.weight'].resize(1,hiddensizes[0]*hiddensizes[1]))
wi3 = np.asarray(wi['4.weight'].resize(1,hiddensizes[1]*hiddensizes[2]))
wi4 = np.asarray(wi['6.weight'].resize(1,hiddensizes[2]*10))
w_init = np.concatenate((wi1,wi2,wi3,wi4), axis = 1)
w_init = w_init.T

w_init.ravel()
w_init.squeeze()

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  A_large = sparse.random(dim_w,dim_w, format='csr', density=den, data_rvs = rvs )

  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      A = A_large[0:n_rows,:]
      w = np.zeros(dim_w)


      w_diff = csr_matrix(wf_vec - w_init)
      b_eq = sparse.csr_matrix.dot(A,w_diff)

      bn = b_eq.toarray()
      bn.reshape((-1,))
      
      sol = lsqr(A,bn)
      z = sol[0]
      z = np.asmatrix(z).T
      w = z+ w_init

      #test performance of this reinitialization
      w1= w[0:hiddensizes[0]*3072]
      w1 = w1.reshape(hiddensizes[0] , 3072)
      w2 = w[hiddensizes[0]*3072:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*3072]
      w2 = w2.reshape(hiddensizes[1],hiddensizes[0])
      w3 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*3072:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*3072+hiddensizes[1]*hiddensizes[2]]
      w3 = w3.reshape(hiddensizes[2],hiddensizes[1])
      w4 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*3072+hiddensizes[1]*hiddensizes[2]: hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*3072+hiddensizes[1]*hiddensizes[2] +hiddensizes[2]*10]
      w4 = w4.reshape(10 , hiddensizes[2])

      wf =torch.load('weights_f.pth')
      wf['0.weight'] = torch.from_numpy(w1)
      wf['2.weight'] = torch.from_numpy(w2)
      wf['4.weight'] = torch.from_numpy(w3)
      wf['6.weight'] = torch.from_numpy(w4)
      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct_count, all_count = 0, 0
      for images,labels in testloader:
          for i in range(len(labels)):
              img = images[i].view(1, 3072)
              with torch.no_grad():
                  logps = net(img)


              ps = torch.exp(logps)
              probab = list(ps.numpy()[0])
              pred_label = probab.index(max(probab))
              true_label = labels.numpy()[i]
              if(true_label == pred_label):
                    correct_count += 1
              all_count += 1

      acc[i_out,i_in] = correct_count/all_count
      dist_i[i_out,i_in] = np.linalg.norm(w-w_init)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in]))    

/usr/local/lib/python3.6/dist-packages/torch/tensor.py:365: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


outer iteration number: 1

 n_rows = 2 , Accuracy =0.1021

 Dist from Wi = 0.11849431538604215 , Dist from Wf =59.65313006594129

 n_rows = 4 , Accuracy =0.1025

 Dist from Wi = 0.16035917003713304 , Dist from Wf =59.65303221550519

 n_rows = 8 , Accuracy =0.1027

 Dist from Wi = 0.2506231134683954 , Dist from Wf =59.65272127590301

 n_rows = 16 , Accuracy =0.1024

 Dist from Wi = 0.2894312149669084 , Dist from Wf =59.65254560483393

 n_rows = 32 , Accuracy =0.1006

 Dist from Wi = 0.5182738814766882 , Dist from Wf =59.650996301383294

 n_rows = 64 , Accuracy =0.1001

 Dist from Wi = 0.801228718369873 , Dist from Wf =59.647866685355304

 n_rows = 128 , Accuracy =0.0986

 Dist from Wi = 1.140439595581852 , Dist from Wf =59.64234540251276

 n_rows = 256 , Accuracy =0.0986

 Dist from Wi = 1.4982260307030977 , Dist from Wf =59.634430376632764

 n_rows = 512 , Accuracy =0.097

 Dist from Wi = 2.2048700090170774 , Dist from Wf =59.612486241172554

 n_rows = 1024 , Accuracy =0.0977

 Dist fr

In [ ]:
np.save("accuracy_A.npy", acc)
np.save("distance_i_A.npy", dist_i)
np.save("distance_f_A.npy", dist_f)

In [ ]:
dim_w

427264

In [ ]:
# A+I 
from scipy import sparse
I = sparse.identity(dim_w)
n_runs =2
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384,32768,65536,131072,262144,427264])
den = 10/dim_w
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))
rvs = stats.norm().rvs

wi = torch.load('weights_i.pth')
wi1 = np.asarray(wi['0.weight'].resize(1,hiddensizes[0]*3072))
wi2 = np.asarray(wi['2.weight'].resize(1,hiddensizes[0]*hiddensizes[1]))
wi3 = np.asarray(wi['4.weight'].resize(1,hiddensizes[1]*hiddensizes[2]))
wi4 = np.asarray(wi['6.weight'].resize(1,hiddensizes[2]*10))
w_init = np.concatenate((wi1,wi2,wi3,wi4), axis = 1)
w_init = w_init.T
w_init.ravel()
w_init.squeeze()

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  A_large = sparse.random(dim_w,dim_w, format='csr', density=den, data_rvs = rvs )
  A_large = A_large + I
  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      A = A_large[0:n_rows,:]
      w = np.zeros(dim_w)
 
      w_diff = csr_matrix(wf_vec - w_init)
      b_eq = sparse.csr_matrix.dot(A,w_diff)

      bn = b_eq.toarray()
      bn.reshape((-1,))
      
      sol = lsqr(A,bn)
      z = sol[0]
      z = np.asmatrix(z).T
      w = z+ w_init

      #test performance of this reinitialization
      w1= w[0:hiddensizes[0]*3072]
      w1 = w1.reshape(hiddensizes[0] , 3072)
      w2 = w[hiddensizes[0]*3072:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*3072]
      w2 = w2.reshape(hiddensizes[1],hiddensizes[0])
      w3 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*3072:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*3072+hiddensizes[1]*hiddensizes[2]]
      w3 = w3.reshape(hiddensizes[2],hiddensizes[1])
      w4 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*3072+hiddensizes[1]*hiddensizes[2]: hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*3072+hiddensizes[1]*hiddensizes[2] +hiddensizes[2]*10]
      w4 = w4.reshape(10 , hiddensizes[2])

      wf =torch.load('weights_f.pth')
      wf['0.weight'] = torch.from_numpy(w1)
      wf['2.weight'] = torch.from_numpy(w2)
      wf['4.weight'] = torch.from_numpy(w3)
      wf['6.weight'] = torch.from_numpy(w4)
      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct_count, all_count = 0, 0
      for images,labels in testloader:
          for i in range(len(labels)):
              img = images[i].view(1, 3072)
              with torch.no_grad():
                  logps = net(img)


              ps = torch.exp(logps)
              probab = list(ps.numpy()[0])
              pred_label = probab.index(max(probab))
              true_label = labels.numpy()[i]
              if(true_label == pred_label):
                    correct_count += 1
              all_count += 1

      acc[i_out,i_in] = correct_count/all_count
      dist_i[i_out,i_in] = np.linalg.norm(w-w_init)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in]))    

/usr/local/lib/python3.6/dist-packages/torch/tensor.py:365: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


outer iteration number: 1

 n_rows = 2 , Accuracy =0.1022

 Dist from Wi = 0.10047886502525145 , Dist from Wf =59.65316313126488

 n_rows = 4 , Accuracy =0.1021

 Dist from Wi = 0.19801612553969186 , Dist from Wf =59.65291910023813

 n_rows = 8 , Accuracy =0.103

 Dist from Wi = 0.4617421234216581 , Dist from Wf =59.651460684270894

 n_rows = 16 , Accuracy =0.1031

 Dist from Wi = 0.5278310627988579 , Dist from Wf =59.65091249867448

 n_rows = 32 , Accuracy =0.1031

 Dist from Wi = 0.6646210677681766 , Dist from Wf =59.649545232067524

 n_rows = 64 , Accuracy =0.102

 Dist from Wi = 0.7764389586526712 , Dist from Wf =59.64819452507478

 n_rows = 128 , Accuracy =0.1021

 Dist from Wi = 1.0954603034653507 , Dist from Wf =59.643188498597134

 n_rows = 256 , Accuracy =0.1005

 Dist from Wi = 1.3994625660223248 , Dist from Wf =59.6368298292758

 n_rows = 512 , Accuracy =0.0992

 Dist from Wi = 1.9971022433170311 , Dist from Wf =59.61980837091396

 n_rows = 1024 , Accuracy =0.1007

 Dist fro

In [ ]:
np.save("accuracy_AI.npy", acc)
np.save("distance_i_AI.npy", dist_i)
np.save("distance_f_AI.npy", dist_f)

In [ ]:
#I
from scipy import sparse
n_runs =1
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384,32768,65536,131072,262144,3072*128,3072*128+128*128,3072*128+128*128+128*128,3072*128+128*128+128*128+1280])
num_rows_arr = np.sort(num_rows_arr)

den = 10/dim_w
I = sparse.identity(dim_w)
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))
rvs = stats.norm().rvs

wi = torch.load('weights_i.pth')
wi1 = np.asarray(wi['0.weight'].resize(1,hiddensizes[0]*inputsize))
wi2 = np.asarray(wi['2.weight'].resize(1,hiddensizes[0]*hiddensizes[1]))
wi3 = np.asarray(wi['4.weight'].resize(1,hiddensizes[1]*hiddensizes[2]))
wi4 = np.asarray(wi['6.weight'].resize(1,hiddensizes[2]*10))
w_init = np.concatenate((wi1,wi2,wi3,wi4), axis = 1)
w_init = w_init.T

w_init.ravel()
w_init.squeeze()

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  for i_in in range(num_rows_arr.shape[0]):
      
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      w = np.zeros((dim_w,1))
      w[0:n_rows] = wf_vec[0:n_rows]
      w[n_rows:] = w_init[n_rows:] 

      #test performance of this reinitialization
      w1= w[0:hiddensizes[0]*inputsize]
      w1 = w1.reshape(hiddensizes[0] , inputsize)
      w2 = w[hiddensizes[0]*inputsize:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize]
      w2 = w2.reshape(hiddensizes[1],hiddensizes[0])
      w3 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize+hiddensizes[1]*hiddensizes[2]]
      w3 = w3.reshape(hiddensizes[2],hiddensizes[1])
      w4 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize+hiddensizes[1]*hiddensizes[2]: hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize+hiddensizes[1]*hiddensizes[2] +hiddensizes[2]*10]
      w4 = w4.reshape(10 , hiddensizes[2])

      wf =torch.load('weights_f.pth')
      wf['0.weight'] = torch.from_numpy(w1)
      wf['2.weight'] = torch.from_numpy(w2)
      wf['4.weight'] = torch.from_numpy(w3)
      wf['6.weight'] = torch.from_numpy(w4)
      torch.save(wf,'w_affex.pth')

      net.load_state_dict(torch.load('w_affex.pth'))

      #testing
      correct_count, all_count = 0, 0
      for images,labels in testloader:
          for i in range(len(labels)):
              img = images[i].view(1, inputsize)
              with torch.no_grad():
                  logps = net(img)

              ps = torch.exp(logps)
              probab = list(ps.numpy()[0])
              pred_label = probab.index(max(probab))
              true_label = labels.numpy()[i]
              if(true_label == pred_label):
                    correct_count += 1
              all_count += 1

      acc[i_out,i_in] = correct_count/all_count
      dist_i[i_out,i_in] = np.linalg.norm(w-w_init)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in]))    

outer iteration number: 1


/usr/local/lib/python3.6/dist-packages/torch/tensor.py:365: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")



 n_rows = 2 , Accuracy =0.1023

 Dist from Wi = 0.11375125690359036 , Dist from Wf =59.65313929894045

 n_rows = 4 , Accuracy =0.1024

 Dist from Wi = 0.1223575181890343 , Dist from Wf =59.65312226702793

 n_rows = 8 , Accuracy =0.1023

 Dist from Wi = 0.13628933464823997 , Dist from Wf =59.653092063903124

 n_rows = 16 , Accuracy =0.1023

 Dist from Wi = 0.17461172145053333 , Dist from Wf =59.652992199167784

 n_rows = 32 , Accuracy =0.1019

 Dist from Wi = 0.23735743573686288 , Dist from Wf =59.65277553488137

 n_rows = 64 , Accuracy =0.1009

 Dist from Wi = 0.34352460689160724 , Dist from Wf =59.652258619533434

 n_rows = 128 , Accuracy =0.1012

 Dist from Wi = 0.45699152474236504 , Dist from Wf =59.65149726799449

 n_rows = 256 , Accuracy =0.1014

 Dist from Wi = 0.5842677407857655 , Dist from Wf =59.65038640926242

 n_rows = 512 , Accuracy =0.1

 Dist from Wi = 1.1515239388758496 , Dist from Wf =59.64213242486755

 n_rows = 1024 , Accuracy =0.1023

 Dist from Wi = 1.7014393684498

In [ ]:
np.save("accuracy_I.npy", acc)
np.save("distance_i_I.npy", dist_i)
np.save("distance_f_I.npy", dist_f)

In [ ]:
#Irev
from scipy import sparse
n_runs =1
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384,32768,65536,131072,262144,3072*128,3072*128+128*128,3072*128+128*128+128*128,3072*128+128*128+128*128+1280])
num_rows_arr = np.sort(num_rows_arr)

den = 10/dim_w
I = sparse.identity(dim_w)
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))
rvs = stats.norm().rvs

wi = torch.load('weights_i.pth')
wi1 = np.asarray(wi['0.weight'].resize(1,hiddensizes[0]*inputsize))
wi2 = np.asarray(wi['2.weight'].resize(1,hiddensizes[0]*hiddensizes[1]))
wi3 = np.asarray(wi['4.weight'].resize(1,hiddensizes[1]*hiddensizes[2]))
wi4 = np.asarray(wi['6.weight'].resize(1,hiddensizes[2]*10))
w_init = np.concatenate((wi1,wi2,wi3,wi4), axis = 1)
w_init = w_init.T

w_init.ravel()
w_init.squeeze()

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  for i_in in range(num_rows_arr.shape[0]):
      
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      w = np.zeros((dim_w,1))
      w[0:-n_rows] = w_init[0:-n_rows]
      w[-n_rows:] = wf_vec[-n_rows:] 

      #test performance of this reinitialization
      w1= w[0:hiddensizes[0]*inputsize]
      w1 = w1.reshape(hiddensizes[0] , inputsize)
      w2 = w[hiddensizes[0]*inputsize:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize]
      w2 = w2.reshape(hiddensizes[1],hiddensizes[0])
      w3 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize+hiddensizes[1]*hiddensizes[2]]
      w3 = w3.reshape(hiddensizes[2],hiddensizes[1])
      w4 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize+hiddensizes[1]*hiddensizes[2]: hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize+hiddensizes[1]*hiddensizes[2] +hiddensizes[2]*10]
      w4 = w4.reshape(10 , hiddensizes[2])

      wf =torch.load('weights_f.pth')
      wf['0.weight'] = torch.from_numpy(w1)
      wf['2.weight'] = torch.from_numpy(w2)
      wf['4.weight'] = torch.from_numpy(w3)
      wf['6.weight'] = torch.from_numpy(w4)
      torch.save(wf,'w_affex.pth')

      net.load_state_dict(torch.load('w_affex.pth'))

      #testing
      correct_count, all_count = 0, 0
      for images,labels in testloader:
          for i in range(len(labels)):
              img = images[i].view(1, inputsize)
              with torch.no_grad():
                  logps = net(img)

              ps = torch.exp(logps)
              probab = list(ps.numpy()[0])
              pred_label = probab.index(max(probab))
              true_label = labels.numpy()[i]
              if(true_label == pred_label):
                    correct_count += 1
              all_count += 1

      acc[i_out,i_in] = correct_count/all_count
      dist_i[i_out,i_in] = np.linalg.norm(w-w_init)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in]))    

outer iteration number: 1


/usr/local/lib/python3.6/dist-packages/torch/tensor.py:365: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")



 n_rows = 2 , Accuracy =0.1022

 Dist from Wi = 0.26600348556372005 , Dist from Wf =59.65265467448124

 n_rows = 4 , Accuracy =0.1022

 Dist from Wi = 0.30956641179918754 , Dist from Wf =59.65244451155314

 n_rows = 8 , Accuracy =0.1022

 Dist from Wi = 0.31296086768757997 , Dist from Wf =59.652426799439915

 n_rows = 16 , Accuracy =0.1022

 Dist from Wi = 0.5984273377307718 , Dist from Wf =59.65024603711788

 n_rows = 32 , Accuracy =0.1022

 Dist from Wi = 0.9239399949321957 , Dist from Wf =59.646092097077116

 n_rows = 64 , Accuracy =0.1022

 Dist from Wi = 1.2651936619096922 , Dist from Wf =59.63982941428576

 n_rows = 128 , Accuracy =0.1022

 Dist from Wi = 1.7720155851095791 , Dist from Wf =59.62692284810083

 n_rows = 256 , Accuracy =0.1022

 Dist from Wi = 2.36888147462735 , Dist from Wf =59.60619404161289

 n_rows = 512 , Accuracy =0.1007

 Dist from Wi = 3.1250366272377565 , Dist from Wf =59.571336342620874

 n_rows = 1024 , Accuracy =0.1021

 Dist from Wi = 4.491576976437761

In [ ]:
np.save("accuracy_UI.npy", acc)
np.save("distance_i_UI.npy", dist_i)
np.save("distance_f_UI.npy", dist_f)

In [ ]:
from google.colab import files
files.download('accuracy_UI.npy') 
files.download('distance_i_UI.npy')
files.download('distance_f_UI.npy')

files.download('accuracy_I.npy') 
files.download('distance_i_I.npy')
files.download('distance_f_I.npy')

files.download('accuracy_AI.npy') 
files.download('distance_i_AI.npy')
files.download('distance_f_AI.npy')

files.download('accuracy_A.npy') 
files.download('distance_i_A.npy')
files.download('distance_f_A.npy')

In [ ]:
#the following blocks are for experimenting with a 256 node FCN network which was later dropped because too high dimensional
#current net is 256 node architecture 
#w_init run number 3
from scipy import sparse
I = sparse.identity(dim_w)
n_runs =1
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384,32768,65536,131072,262144])
den = 10/dim_w
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))
rvs = stats.norm().rvs

wi = torch.load('weights_i.pth')
wi1 = np.asarray(wi['0.weight'].resize(1,hiddensizes[0]*3072))
wi2 = np.asarray(wi['2.weight'].resize(1,hiddensizes[0]*hiddensizes[1]))
wi3 = np.asarray(wi['4.weight'].resize(1,hiddensizes[1]*hiddensizes[2]))
wi4 = np.asarray(wi['6.weight'].resize(1,hiddensizes[2]*10))
w_init = np.concatenate((wi1,wi2,wi3,wi4), axis = 1)
w_init = w_init.T
w_init.ravel()
w_init.squeeze()

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  A_large =  I
  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      A = A_large.tocsr()[0:n_rows,:]
      w = np.zeros(dim_w)
 
      w_diff = csr_matrix(wf_vec - w_init)
      b_eq = sparse.csr_matrix.dot(A,w_diff)

      bn = b_eq.toarray()
      bn.reshape((-1,))
      
      sol = lsqr(A,bn)
      z = sol[0]
      z = np.asmatrix(z).T
      w = z+ w_init

      #test performance of this reinitialization
      w1= w[0:hiddensizes[0]*3072]
      w1 = w1.reshape(hiddensizes[0] , 3072)
      w2 = w[hiddensizes[0]*3072:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*3072]
      w2 = w2.reshape(hiddensizes[1],hiddensizes[0])
      w3 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*3072:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*3072+hiddensizes[1]*hiddensizes[2]]
      w3 = w3.reshape(hiddensizes[2],hiddensizes[1])
      w4 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*3072+hiddensizes[1]*hiddensizes[2]: hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*3072+hiddensizes[1]*hiddensizes[2] +hiddensizes[2]*10]
      w4 = w4.reshape(10 , hiddensizes[2])

      wf =torch.load('weights_f.pth')
      wf['0.weight'] = torch.from_numpy(w1)
      wf['2.weight'] = torch.from_numpy(w2)
      wf['4.weight'] = torch.from_numpy(w3)
      wf['6.weight'] = torch.from_numpy(w4)
      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct_count, all_count = 0, 0
      for images,labels in testloader:
          for i in range(len(labels)):
              img = images[i].view(1, 3072)
              with torch.no_grad():
                  logps = net(img)


              ps = torch.exp(logps)
              probab = list(ps.numpy()[0])
              pred_label = probab.index(max(probab))
              true_label = labels.numpy()[i]
              if(true_label == pred_label):
                    correct_count += 1
              all_count += 1

      acc[i_out,i_in] = correct_count/all_count
      dist_i[i_out,i_in] = np.linalg.norm(w-w_init)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in]))    

outer iteration number: 1


/usr/local/lib/python3.6/dist-packages/torch/tensor.py:362: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")



 n_rows = 2 , Accuracy =0.1

 Dist from Wi = 0.0888002413052202 , Dist from Wf =66.57442847891889

 n_rows = 4 , Accuracy =0.1

 Dist from Wi = 0.13789669125572598 , Dist from Wf =66.57434488810657

 n_rows = 8 , Accuracy =0.1

 Dist from Wi = 0.1606201412431115 , Dist from Wf =66.57429394254237

 n_rows = 16 , Accuracy =0.1

 Dist from Wi = 0.20473537334108158 , Dist from Wf =66.5741728916288

 n_rows = 32 , Accuracy =0.1

 Dist from Wi = 0.2828343295719274 , Dist from Wf =66.57388690410549

 n_rows = 64 , Accuracy =0.1

 Dist from Wi = 0.45806582163299386 , Dist from Wf =66.57291182216808

 n_rows = 128 , Accuracy =0.1

 Dist from Wi = 0.6339083814472608 , Dist from Wf =66.57146966189286

 n_rows = 256 , Accuracy =0.1

 Dist from Wi = 1.0628372064267582 , Dist from Wf =66.5660032587852

 n_rows = 512 , Accuracy =0.1

 Dist from Wi = 1.6498633392797806 , Dist from Wf =66.55404092727117

 n_rows = 1024 , Accuracy =0.1

 Dist from Wi = 2.772955585623432 , Dist from Wf =66.5167131636489

In [ ]:
np.save("accuracy_I3.npy", acc)
np.save("distance_i_I3.npy", dist_i)
np.save("distance_f_I3.npy", dist_f)

In [ ]:
#I UI experiment

array([0, 1, 3, 4, 5, 6, 7, 8])

In [ ]:
!pip install q scipy==1.1.0
import numpy as np
import torch 
import matplotlib.pyplot as plt
import torchvision
from time import time 
from torchvision import datasets, transforms
from torch import nn,optim 
from scipy.sparse.linalg import lsqr
from scipy.sparse import csr_matrix
from scipy.sparse import random
from scipy.sparse import rand
from scipy import stats


     |████████████████████████████████| 31.2MB 104kB/s 
ERROR: umap-learn 0.4.3 has requirement scipy>=1.3.1, but you'll have scipy 1.1.0 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement scipy==1.4.1; python_version >= "3", but you'll have scipy 1.1.0 which is incompatible.
ERROR: plotnine 0.6.0 has requirement scipy>=1.2.0, but you'll have scipy 1.1.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1


In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)


#define the model 
inputsize = 3072
hiddensizes = [128,128,128]
outputsize = 10

criterion = nn.NLLLoss()
images, labels = next(iter(trainloader))
images = images.view(images.shape[0],-1)


net = nn.Sequential(nn.Linear(inputsize,hiddensizes[0]),nn.ReLU(),nn.Linear(hiddensizes[0],hiddensizes[1]),nn.ReLU(),
                    nn.Linear(hiddensizes[1],hiddensizes[2]),nn.ReLU(),nn.Linear(hiddensizes[2],outputsize),nn.LogSoftmax(dim=1))
net.apply(init_weights)

w_init= list(net.parameters())
torch.save(net.state_dict(),'weights_i.pth')

print(net)
logps = (net(images))
loss = criterion(logps,labels)


optimizer = optim.SGD(net.parameters(),lr = 0.003, momentum = 0.9 )


Sequential(
  (0): Linear(in_features=3072, out_features=128, bias=True)
  (1): ReLU()
  (2): Linear(in_features=128, out_features=128, bias=True)
  (3): ReLU()
  (4): Linear(in_features=128, out_features=128, bias=True)
  (5): ReLU()
  (6): Linear(in_features=128, out_features=10, bias=True)
  (7): LogSoftmax()
)


In [ ]:
epochs =15
time0 = time()
for e in range(epochs):
    running_loss = 0
    for images,labels in trainloader:
        images = images.view(images.shape[0],-1)
        optimizer.zero_grad()
        output = net(images)
        loss = criterion(output,labels)
        loss.backward()
        optimizer.step()
        running_loss+=loss.item()
#     else:
    print("Epoch {} - Training loss: {}".format(e, running_loss/len(trainloader)))
#     print(running_loss)
            
    print("\n Training time in minutes = ", (time()-time0)/60)

Epoch 0 - Training loss: 1.738986527172327

 Training time in minutes =  1.1460038622220357
Epoch 1 - Training loss: 1.588506613036394

 Training time in minutes =  2.2944128354390463
Epoch 2 - Training loss: 1.5361389113515616

 Training time in minutes =  3.4438008944193523
Epoch 3 - Training loss: 1.4992918165130913

 Training time in minutes =  4.605566477775573
Epoch 4 - Training loss: 1.4669616214787216

 Training time in minutes =  5.75686225493749
Epoch 5 - Training loss: 1.4440978489421308

 Training time in minutes =  6.931402409076691
Epoch 6 - Training loss: 1.421856378351003

 Training time in minutes =  8.098209118843078
Epoch 7 - Training loss: 1.4020845537522435

 Training time in minutes =  9.28338150580724
Epoch 8 - Training loss: 1.3901891127052157

 Training time in minutes =  10.451679488023123
Epoch 9 - Training loss: 1.373728793632388

 Training time in minutes =  11.6192418217659
Epoch 10 - Training loss: 1.3665199653072655

 Training time in minutes =  12.77448

In [ ]:
#testing

correct_count, all_count = 0, 0
for images,labels in testloader:
    for i in range(len(labels)):
        img = images[i].view(1, 3072)
        with torch.no_grad():
            logps = net(img)
        ps = torch.exp(logps)
        probab = list(ps.numpy()[0])
        pred_label = probab.index(max(probab))
        true_label = labels.numpy()[i]
        if(true_label == pred_label):
              correct_count += 1
        all_count += 1
print("Number Of Images Tested =", all_count)
print("\nModel Accuracy =", (correct_count/all_count))
model_accuracy = correct_count/all_count
np.save("model_accuracy_3.npy",model_accuracy)

Number Of Images Tested = 10000

Model Accuracy = 0.47


In [ ]:
torch.save(net.state_dict(),'weights_f.pth')

#Projecting weights on affine spaces defined by the set P: {AW = AW_final} 
#Projection is done using L2 minimization w= argmin ||w-w_initial|| in P

#vectorize W
wf =torch.load('weights_f.pth')
wf1 = np.asarray(wf['0.weight'].resize(1,hiddensizes[0]*3072))
wf2 = np.asarray(wf['2.weight'].resize(1,hiddensizes[0]*hiddensizes[1]))
wf3 = np.asarray(wf['4.weight'].resize(1,hiddensizes[1]*hiddensizes[2]))
wf4 = np.asarray(wf['6.weight'].resize(1,hiddensizes[2]*10))
wf_vec = np.concatenate((wf1,wf2,wf3,wf4),axis =1)
wf_vec = wf_vec.T
dim_w = wf_vec.shape[0]

/usr/local/lib/python3.6/dist-packages/torch/tensor.py:365: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


In [ ]:
dim_w

427264

In [ ]:
3072*128+ 128*128+128*128+1280

427264

In [ ]:
#A = I run 1 
#128 node architecture
#w_init run number 3
from scipy import sparse
n_runs =1
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384,32768,65536,131072,3072*128,128*128,128*128,1280])
num_rows_arr = np.sort(num_rows_arr)
den = 10/dim_w
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))
rvs = stats.norm().rvs
I = sparse.identity(dim_w).tocsr()

wi = torch.load('weights_i.pth')
wi1 = np.asarray(wi['0.weight'].resize(1,hiddensizes[0]*3072))
wi2 = np.asarray(wi['2.weight'].resize(1,hiddensizes[0]*hiddensizes[1]))
wi3 = np.asarray(wi['4.weight'].resize(1,hiddensizes[1]*hiddensizes[2]))
wi4 = np.asarray(wi['6.weight'].resize(1,hiddensizes[2]*10))
w_init = np.concatenate((wi1,wi2,wi3,wi4), axis = 1)
w_init = w_init.T

w_init.ravel()
w_init.squeeze()

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  A_large = I

  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      A = A_large[0:n_rows,:]
      w = np.zeros(dim_w)


      w_diff = csr_matrix(wf_vec - w_init)
      b_eq = sparse.csr_matrix.dot(A,w_diff)

      bn = b_eq.toarray()
      bn.reshape((-1,))
      
      sol = lsqr(A,bn)
      z = sol[0]
      z = np.asmatrix(z).T
      w = z+ w_init

      #test performance of this reinitialization
      w1= w[0:hiddensizes[0]*3072]
      w1 = w1.reshape(hiddensizes[0] , 3072)
      w2 = w[hiddensizes[0]*3072:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*3072]
      w2 = w2.reshape(hiddensizes[1],hiddensizes[0])
      w3 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*3072:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*3072+hiddensizes[1]*hiddensizes[2]]
      w3 = w3.reshape(hiddensizes[2],hiddensizes[1])
      w4 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*3072+hiddensizes[1]*hiddensizes[2]: hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*3072+hiddensizes[1]*hiddensizes[2] +hiddensizes[2]*10]
      w4 = w4.reshape(10 , hiddensizes[2])

      wf =torch.load('weights_f.pth')
      wf['0.weight'] = torch.from_numpy(w1)
      wf['2.weight'] = torch.from_numpy(w2)
      wf['4.weight'] = torch.from_numpy(w3)
      wf['6.weight'] = torch.from_numpy(w4)
      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct_count, all_count = 0, 0
      for images,labels in testloader:
          for i in range(len(labels)):
              img = images[i].view(1, 3072)
              with torch.no_grad():
                  logps = net(img)


              ps = torch.exp(logps)
              probab = list(ps.numpy()[0])
              pred_label = probab.index(max(probab))
              true_label = labels.numpy()[i]
              if(true_label == pred_label):
                    correct_count += 1
              all_count += 1

      acc[i_out,i_in] = correct_count/all_count
      dist_i[i_out,i_in] = np.linalg.norm(w-w_init)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in]))    

outer iteration number: 1


/usr/local/lib/python3.6/dist-packages/torch/tensor.py:365: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")



 n_rows = 2 , Accuracy =0.1013

 Dist from Wi = 0.1631060892898546 , Dist from Wf =67.65511652808294

 n_rows = 4 , Accuracy =0.1013

 Dist from Wi = 0.16673383720750531 , Dist from Wf =67.65510768487754

 n_rows = 8 , Accuracy =0.1013

 Dist from Wi = 0.1700083769245483 , Dist from Wf =67.65509953563456

 n_rows = 16 , Accuracy =0.1012

 Dist from Wi = 0.1969439592641129 , Dist from Wf =67.65502648807316

 n_rows = 32 , Accuracy =0.1011

 Dist from Wi = 0.27729950095240946 , Dist from Wf =67.65474485216988

 n_rows = 64 , Accuracy =0.1008

 Dist from Wi = 0.34676794444755576 , Dist from Wf =67.65442445264561

 n_rows = 128 , Accuracy =0.1007

 Dist from Wi = 0.5254405972983669 , Dist from Wf =67.65327270875859

 n_rows = 256 , Accuracy =0.1007

 Dist from Wi = 0.7995304055181645 , Dist from Wf =67.65058866822017

 n_rows = 512 , Accuracy =0.0971

 Dist from Wi = 1.7198168222949675 , Dist from Wf =67.63345049697257

 n_rows = 1024 , Accuracy =0.0939

 Dist from Wi = 2.4463273920095827

In [ ]:
np.save("accuracy_I1.npy", acc)
np.save("distance_i_I1.npy", dist_i)
np.save("distance_f_I1.npy", dist_f)

In [ ]:
#A = I run 1 
#128 node architecture
#w_init run number 3
from scipy import sparse
n_runs =1
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384,32768,65536,131072,3072*128,128*128,128*128,1280])
num_rows_arr = np.sort(num_rows_arr)
den = 10/dim_w
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))
rvs = stats.norm().rvs
I = sparse.identity(dim_w).tocsr()
Irev = I[::-1] 
wi = torch.load('weights_i.pth')
wi1 = np.asarray(wi['0.weight'].resize(1,hiddensizes[0]*3072))
wi2 = np.asarray(wi['2.weight'].resize(1,hiddensizes[0]*hiddensizes[1]))
wi3 = np.asarray(wi['4.weight'].resize(1,hiddensizes[1]*hiddensizes[2]))
wi4 = np.asarray(wi['6.weight'].resize(1,hiddensizes[2]*10))
w_init = np.concatenate((wi1,wi2,wi3,wi4), axis = 1)
w_init = w_init.T

w_init.ravel()
w_init.squeeze()

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  A_large = Irev

  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      A = A_large[0:n_rows,:]
      w = np.zeros(dim_w)


      w_diff = csr_matrix(wf_vec - w_init)
      b_eq = sparse.csr_matrix.dot(A,w_diff)

      bn = b_eq.toarray()
      bn.reshape((-1,))
      
      sol = lsqr(A,bn)
      z = sol[0]
      z = np.asmatrix(z).T
      w = z+ w_init

      #test performance of this reinitialization
      w1= w[0:hiddensizes[0]*3072]
      w1 = w1.reshape(hiddensizes[0] , 3072)
      w2 = w[hiddensizes[0]*3072:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*3072]
      w2 = w2.reshape(hiddensizes[1],hiddensizes[0])
      w3 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*3072:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*3072+hiddensizes[1]*hiddensizes[2]]
      w3 = w3.reshape(hiddensizes[2],hiddensizes[1])
      w4 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*3072+hiddensizes[1]*hiddensizes[2]: hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*3072+hiddensizes[1]*hiddensizes[2] +hiddensizes[2]*10]
      w4 = w4.reshape(10 , hiddensizes[2])

      wf =torch.load('weights_f.pth')
      wf['0.weight'] = torch.from_numpy(w1)
      wf['2.weight'] = torch.from_numpy(w2)
      wf['4.weight'] = torch.from_numpy(w3)
      wf['6.weight'] = torch.from_numpy(w4)
      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct_count, all_count = 0, 0
      for images,labels in testloader:
          for i in range(len(labels)):
              img = images[i].view(1, 3072)
              with torch.no_grad():
                  logps = net(img)


              ps = torch.exp(logps)
              probab = list(ps.numpy()[0])
              pred_label = probab.index(max(probab))
              true_label = labels.numpy()[i]
              if(true_label == pred_label):
                    correct_count += 1
              all_count += 1

      acc[i_out,i_in] = correct_count/all_count
      dist_i[i_out,i_in] = np.linalg.norm(w-w_init)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in]))    

In [ ]:
np.save("accuracy_UI1.npy", acc)
np.save("distance_i_UI1.npy", dist_i)
np.save("distance_f_II1.npy", dist_f)